In [12]:
using NBInclude
@nbinclude("dcj_datastructs.ipynb")

genome_to_adj_listset (generic function with 1 method)

In [13]:
# process adj list 
# create dictionary of gene ID --> index/location of gene's head and tail 

function process_adj_list_helper(ge::GeneEnd, idx::Ref{Int}, gid_to_loc::DefaultDict{Int, Vector{Int}})
    if ge.gene == Telomere() 
        return
    end  

    if ge.head == true
        gid_to_loc[ge.gene.id][2] = idx[]
    else 
        gid_to_loc[ge.gene.id][1] = idx[]
    end
end 

function process_adj_list(adj_list:: Vector{Adjacency})
    geneid_to_location = DefaultDict{Int, Vector{Int}}(() -> zeros(Int, 2)) # tail = idx 1, head = 2 in array
    idx = Ref{Int}(1)

    for adj in adj_list
        process_adj_list_helper(adj.left, idx, geneid_to_location)
        process_adj_list_helper(adj.right, idx, geneid_to_location)
        idx[] += 1
    end 
    
    return geneid_to_location 
end 

process_adj_list (generic function with 1 method)

In [14]:
# helpers for dcj operations and distance

# finds target gene end in source adjacency list 
# returns index and left/right in the source adj list
function find_tar_ge_in_src_adjlist(target_ge::GeneEnd, src_gid_to_l::DefaultDict{Int, Vector{Int}}, tar_gid_to_l::DefaultDict{Int, Vector{Int}}, src_adj_list::Vector{Adjacency})
    (target_ge.head == true) ? th_idx = 2 : th_idx = 1
    # (left_tar_ge == true) ? tar_lr = 'L' : tar_lr = 'R'
        
    gene_id = target_ge.gene.id

    src_ge_idx = src_gid_to_l[gene_id][th_idx]

    adj = src_adj_list[src_ge_idx]
    if adj.left.gene.id == gene_id
        src_ge = adj.left
        # src_lr = 'L'
    else 
        src_ge = adj.right 
        # src_lr = 'R' 
    end 

    # print("\n   GENEEND: ")
    # show(target_ge)
    # print("\n     src genome idx: ", src_ge_idx, " ", src_lr)
    # print("\n     target genome idx: ", tar_gid_to_l[gene_id][th_idx], " ", tar_lr, "\n")

    return [src_ge, src_ge_idx]
end 

function other_adjacency_end(ge::GeneEnd, adj::Adjacency)
    if adj.left == ge
        return adj.right
    else
        return adj.left
    end 
end

function non_telo_end(adj::Adjacency)
    if adj.left.gene == Telomere()
        return adj.right
    else
        return adj.left
    end 
end

function combine_ge(u::GeneEnd, u_idx::Int, v::GeneEnd, v_idx::Int)
    if u_idx < v_idx 
        u_lt_v = true 
    else 
        u_lt_v = false 
    end 

    if !u_lt_v 
        return Adjacency(v, u)
    else 
        return Adjacency(u, v)
    end

end 

function reassign_locs(gid_to_l, adj_list, u_idx, v_idx)
    for (i, adj) in pairs(view(adj_list, min(u_idx, v_idx):max(u_idx, v_idx)))
        left = adj.left 
        right = adj.right 

        idx = Ref{Int}(i + min(u_idx, v_idx) - 1)

        process_adj_list_helper(left, idx, gid_to_l)
        process_adj_list_helper(right, idx, gid_to_l)
    end 
end 



reassign_locs (generic function with 1 method)

In [15]:
function update_adj_set(p::GeneEnd, q::GeneEnd, u::GeneEnd, v::GeneEnd, u_idx::Int, v_idx::Int, src_adj_list::Vector{Adjacency}, src_adj_set::Set{Adjacency}) 
    #  replace adj u and v in A by ( {p, q} and (u\{p}) U (v\{q}) )
    pq = combine_ge(p, u_idx, q, v_idx)

    other_ge_u = other_adjacency_end(u, src_adj_list[u_idx])
    other_ge_v = other_adjacency_end(v, src_adj_list[v_idx])
    excluding_pq = combine_ge(other_ge_u, u_idx, other_ge_v, v_idx) 
    
    # print("******exlcuding pq")
    # show(excluding_pq)
    # print("********")

    #  remove adj with u, v in src_adj_set 
    delete!(src_adj_set, src_adj_list[u_idx])
    delete!(src_adj_set, src_adj_list[v_idx])
    
    #  add adjusted adjacencies to src_adj_set
    push!(src_adj_set, pq)
    push!(src_adj_set, excluding_pq)

    # print("  src_adj_set:")
    # show(src_adj_set)
    # print("\n")
end 

function update_adj_list_gid_loc_dict() 
    # update adj lists 
        print("\nDCJ Operation (Cycle) :: ")
        show(src_adj_list, u_idx, v_idx)
        
        print("\n")
        show(pq)
        show(excluding_pq)
        if u_idx < v_idx 
            src_adj_list[u_idx] = pq
            src_adj_list[v_idx] = excluding_pq
            reverse!(view(src_adj_list, u_idx+1:v_idx-1))
        else 
            src_adj_list[u_idx] = excluding_pq
            src_adj_list[v_idx] = pq
            reverse!(view(src_adj_list, v_idx+1:u_idx-1))
        end  
    
        # update gid-->loc dicts 
        reassign_locs(src_gid_to_l, src_adj_list, u_idx, v_idx)
end 

# find dcj operations and distance

function find_dcj_dist_ops(src_adj_list::Vector{Adjacency}, tar_adj_list::Vector{Adjacency}, src_gid_to_l::DefaultDict{Int, Vector{Int}}, tar_gid_to_l::DefaultDict{Int, Vector{Int}}, src_adj_set::Set{Adjacency}, target_adj_set::Set{Adjacency})
    count = 0  
    # cycles = 0 
    # odd_paths = 0 
    telomere_idxs = Vector{Int}

    print("indexing through target adj list...")
    
    # for each adj {p, q} in target genome 
    for (i, adj) in pairs(tar_adj_list) 
        print("\n\n")
        show(src_adj_list)
         
        print("\n\nADJ ", i, " :::: ")
        show(adj)
        print("\n")

        p = adj.left  
        q = adj.right  

        if p.gene == Telomere() || q.gene == Telomere()  # telomeres handled in next loop
            push!(telomere_idxs, i)  
        end 

        #  let u be element of genome A that contains p 
        #  let v be element of genome A that contains q 
        u, u_idx = find_tar_ge_in_src_adjlist(p, src_gid_to_l, tar_gid_to_l, src_adj_list)  
        v, v_idx = find_tar_ge_in_src_adjlist(q, src_gid_to_l, tar_gid_to_l, src_adj_list)  
        
        #  if u != v, replace u and v in A by {p, q} and (u\{p}) U (v\{q})
        if u_idx != v_idx             
            update_adj_set(p, q, u, v, u_idx, v_idx, src_adj_list, src_adj_set)
           
            print("updated adj set ::: ")
            show(src_adj_set)
            print("\nupdated adj list ::: ")
            print_adj_set_as_adj_list(src_adj_set)

            #  update_adj_list_gid_loc_dict()
            
            count += 1 
        end   
    end 

    # for each telomere {p} in genome B 
    # for idx in telomere_idxs
    #     telo_adj = tar_adj_list[idx]
    #     telo_ge = other_adjacency_end(telo_adj, Telomere())

    #     #  let u be element of genome A that contains p 
    #     u, u_idx = find_tar_ge_in_src_adjlist(p, src_gid_to_l, tar_gid_to_l, src_adj_list)  

    #     #  if u is an adjacency, then replace u in A by {p} and (u\{p})
    #     if other_adjacency_end(tar_adj_list[u_idx], u) != Telomere() 
    #         # update_adj_set(p, u, u_idx,src_adj_list, src_adj_set)
    #         # update_adj_list_gid_loc_dict()

    #         count += 1 
    #     end 
    # end 

    return count 
    # return cycles, odd_paths
end 

# src = "abc"
# target = "cba"

# calculate_distance(src, target)

find_dcj_dist_ops (generic function with 1 method)

In [29]:
# given ge we're looking for 
# finds new ge that corresponds to given ge (tail/head) 
# returns new ge in new adjacency 
function find_next_adj(ge::GeneEnd, adj_set::Set{Adjacency}, adj_list::Vector{Adjacency}, processed_adjs::Set{Adjacency}, adj::Adjacency) 
    for adj in adj_set 
        if adj in processed_adjs
            continue 
        end 

        if adj.left.gene == ge.gene
            println("111111111111111111111111111111")
            push!(adj_list, adj)
            push!(processed_adjs, adj)
            ge = other_adjacency_end(adj.left, adj)

            show(adj)
            return ge 
        elseif adj.right.gene == ge.gene
            println("22222222222222222222222")
            push!(adj_list, adj)
            push!(processed_adjs, adj)
            ge = other_adjacency_end(adj.right, adj)

            show(adj)
            return ge 
        end 
    end 
    println("3333333333333333333333333333333")
    # push!(adj_list, adj)
    return other_adjacency_end(ge, adj)
end 

# O(N^2)
function print_adj_set_as_adj_list(og_adj_set::Set{Adjacency})
    telos = Vector{Adjacency}() 
    adj_set = copy(og_adj_set)
    adj_list = Vector{Adjacency}() 

    # handle linear chroms first 
    for adj in adj_set
        # show(adj)
        if adj.left.gene == Telomere() 
            push!(telos, adj)
        end 
    end 

    for adj in telos 
        # ge = adj.right  
        ge = non_telo_end(adj) 
        push!(adj_list, adj)
        delete!(adj_set, adj)

        while ge.gene != Telomere()  
            show(ge)
            ge = find_next_adj(ge, adj_set, adj_list)
        end 
        show(ge)
    end  
    
    # handle circular chroms 
    processed_adjs = Set{Adjacency}() 
    for adj in adj_set  # only circular left 
        # print("\nprocessing :: ")
        # show(adj)
        # println()

        if adj in processed_adjs
            continue 
        end 
        push!(processed_adjs, adj)
        push!(adj_list, adj)
        
        first_gene = adj.left.gene 
        ge = GeneEnd(adj.right.gene, !adj.right.head)  # ge that we're looking for
        first = true       
        
        while ge.gene != first_gene || first == true  
            # print("LOOKING FOR ")
            # show(ge)
            ge = find_next_adj(ge, adj_set, adj_list, processed_adjs, adj)
            # println()
            first = false 
        end 
        # show(ge)
        # print("updated adj_list ---------")
        # show(adj_list)    
        # println()     
    end 
    print("\n**")
    show(adj_list)
    print("*8") 
end 


src = ".a.bc"
target = "cb.a."

calculate_distance(src, target)

SRC ADJ LIST
(.,a:t)(a:h,.)(b:h,c:t)(c:h,b:t)
|
v
TARGET ADJ LIST
(c:h,b:t)(b:h,c:t)(.,a:t)(a:h,.)

*************
indexing through target adj list...

(.,a:t)(a:h,.)(b:h,c:t)(c:h,b:t)

ADJ 1 :::: (c:h,b:t)


(.,a:t)(a:h,.)(b:h,c:t)(c:h,b:t)

ADJ 2 :::: (b:h,c:t)


(.,a:t)(a:h,.)(b:h,c:t)(c:h,b:t)

ADJ 3 :::: (.,a:t)


LoadError: MethodError: no method matching push!(::Type{Vector{Int64}}, ::Int64)

[0mClosest candidates are:
[0m  push!(::Any, ::Any, [91m::Any[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mabstractarray.jl:3413[24m[39m
[0m  push!(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mabstractarray.jl:3414[24m[39m
[0m  push!([91m::Vector{Any}[39m, ::Any)
[0m[90m   @[39m [90mBase[39m [90m[4marray.jl:1125[24m[39m
[0m  ...


In [17]:
# adjacency graph 

mutable struct AdjacencyGraph
    cycles::Int
    odd_paths::Int
end

function Base.show(dict::DefaultDict{Int, Vector{Int}}, id_to_char::Dict{Int, Char})
    for (key, value) in dict
        # println("$(key.dna) (id:$(key.id)) => $(value)")
        println(id_to_char[key], " $(key) => $(value)")
    end
end 

function AdjacencyGraph(src::Genome, target::Genome)

    # process genomes (not explicitly creating the adj graph)    
    src_adj_list, src_adj_set = genome_to_adj_listset(src)        
    target_adj_list, target_adj_set = genome_to_adj_listset(target) 
    
    print("SRC ADJ LIST\n")
    show(src_adj_list)
    # print("\nSRC ADJ SET\n")
    # show(src_adj_set)
    print("\n|\nv\nTARGET ADJ LIST\n")
    show(target_adj_list)
    # print("\nTARGET ADJ SET\n")
    # show(target_adj_set)
    
    print("\n\n*************\n")

    # print_adj_set_as_adj_list(src_adj_set)
    
    src_geneid_to_location = process_adj_list(src_adj_list)   
    target_geneid_to_location = process_adj_list(target_adj_list)
    
    if src.id_to_char != target.id_to_char
        throw(ArgumentError("ERROR: Source ID to char dict is not equal to target's."))
    end 

    # print("SRC GENE --> LOCATION DICT\n")
    # show(src_geneid_to_location, src.id_to_char)  
    # print("\n\n", "TARGET GENE --> LOCATION DICT\n")
    # show(target_geneid_to_location, src.id_to_char)
    # print("\n*************\n\n")

    # use adj graph representation to find dcj distance and operations 
    # cycles, odd_paths = 
    find_dcj_dist_ops(src_adj_list, target_adj_list, src_geneid_to_location, target_geneid_to_location, src_adj_set, target_adj_set) 
    
    # return AdjacencyGraph(cycles, odd_paths)
end 

"""
assumptions 
genomes A and B have the same letters of the same multiplicity; telomeres don't matter 
no empty chromosomes (2 telomeres ..)
"""

function calculate_distance(src::String, target::String)
    # check_conditions(src, target)

    id_counter = Ref{Int}(1)
    id_to_char = Dict{Int, Char}()
    char_to_id = Dict{Char, Int}()

    target_genome = string_to_genome(target, id_counter, id_to_char, char_to_id, true)
    src_genome = string_to_genome(src, id_counter, id_to_char, char_to_id, false)

    ag = AdjacencyGraph(src_genome, target_genome)

    # dcj_distance = src_genome.num_genes - ag.cycles - (ag.ab_paths / 2)  # or target_genome.num_genes

    # return dcj_distance
end

# src = "a.bcde.f"
# target = "abcdef"

# calculate_distance(src, target)

calculate_distance (generic function with 1 method)